In [1]:
# !pip install deep-translator tqdm

In [2]:
import pandas as pd

# Load the first sheet from the Excel file
df = pd.read_excel("../data/pharmcube_cyclic_peptide_20251225.xlsx", sheet_name=0)

# Display basic info
print(f"Shape: {df.shape}")
df.head(2)


Shape: (2137, 26)


,数据层级,药品名称,靶点,作用机制,研发机构,疾病,全球最高研发阶段,全球最高研发阶段开始日期,全球研发状态,药品类别三,...,疾病领域,中国剂型,NPUID,增补数据,原研机构所在国家/地区,原研机构权益地区,中国最高研发阶段,美国最高研发阶段,中国最高研发阶段开始日期,美国最高研发阶段开始日期
0,药品信息,杆菌肽,not available,多肽类抗生素,Columbia University(原研),感染;细菌感染;接触性皮炎(差异化疾病);葡萄球菌感染,批准上市,1948-07-29,Active,多肽,...,感染领域;皮肤领域,NaN,DR004636,否,美国,全球;中国(内地);中国(港澳台);美国;欧洲;日本;其他,批准上市,批准上市,2008-09-22,1948-07-29
1,疾病信息,杆菌肽,not available,多肽类抗生素,Columbia University(原研),细菌感染,批准上市,NaN,Active,多肽,...,感染领域,NaN,DR004636,否,美国,全球;中国(内地);中国(港澳台);美国;欧洲;日本;其他,NaN,II期临床,NaN,2010-10-18


In [3]:
# Check unique values in the phase column
unique_phases = df['全球最高研发阶段'].unique()
unique_phases

array(['批准上市', 'III期临床', 'II期临床', 'I期临床', '申请上市', 'II/III期临床', '临床前',
       'I/II期临床', '申报临床'], dtype=object)

In [4]:
# Mapping from Chinese phase names to standardized English phases
phase_mapping = {
    '临床前': 'pre-clinical',
    'I期临床': 'phase 1',
    'I/II期临床': 'phase 2',
    'II期临床': 'phase 2',
    'II/III期临床': 'phase 3',
    'III期临床': 'phase 3',
    '申请上市': 'pre-registration',
    '申报临床': 'ind',
    '批准上市': 'approved',
}

# Create the standardized phase column
df['highest_phase_standard'] = df['全球最高研发阶段'].map(phase_mapping)

# Check the mapping results
print("Mapping results:")
print(df[['全球最高研发阶段', 'highest_phase_standard']].drop_duplicates())
print(f"\nUnmapped values: {df[df['highest_phase_standard'].isna()]['全球最高研发阶段'].unique()}")


Mapping results:
      全球最高研发阶段 highest_phase_standard
0         批准上市               approved
2       III期临床                phase 3
3        II期临床                phase 2
4         I期临床                phase 1
35        申请上市       pre-registration
38   II/III期临床                phase 3
79         临床前           pre-clinical
145    I/II期临床                phase 2
324       申报临床                    ind

Unmapped values: []


In [5]:
# Extract global highest phase info from "药品信息" rows for each 药品名称+研发机构 pair
drug_info_df = df[df['数据层级'] == '药品信息'][['药品名称', '研发机构', '全球最高研发阶段', '全球最高研发阶段开始日期']].copy()
drug_info_df = drug_info_df.rename(columns={
    '全球最高研发阶段': 'global_highest_phase_all_indication',
    '全球最高研发阶段开始日期': 'global_highest_phase_all_indication_date'
})

# Merge back to the main dataframe
df = df.merge(drug_info_df, on=['药品名称', '研发机构'], how='left')

df['global_highest_phase_all_indication_standard'] = df['global_highest_phase_all_indication'].map(phase_mapping)


# Verify the results
print(f"New columns added: 'global_highest_phase_all_indication', 'global_highest_phase_all_indication_date'")
print(f"\nSample of the new columns:")
df[['药品名称', '研发机构', '数据层级', 'global_highest_phase_all_indication', 'global_highest_phase_all_indication_date']].head(20)


New columns added: 'global_highest_phase_all_indication', 'global_highest_phase_all_indication_date'

Sample of the new columns:


,药品名称,研发机构,数据层级,global_highest_phase_all_indication,global_highest_phase_all_indication_date
0,杆菌肽,Columbia University(原研),药品信息,批准上市,1948-07-29
1,杆菌肽,Columbia University(原研),疾病信息,批准上市,1948-07-29
2,杆菌肽,Columbia University(原研),疾病信息,批准上市,1948-07-29
3,杆菌肽,Columbia University(原研),疾病信息,批准上市,1948-07-29
4,杆菌肽,Columbia University(原研),疾病信息,批准上市,1948-07-29
5,缩宫素,Travere Therapeutics(原研),药品信息,批准上市,1954-06-30
6,缩宫素,Travere Therapeutics(原研),疾病信息,批准上市,1954-06-30
7,缩宫素,Travere Therapeutics(原研),疾病信息,批准上市,1954-06-30
8,缩宫素,Travere Therapeutics(原研),疾病信息,批准上市,1954-06-30
9,多粘菌素B,--,药品信息,批准上市,1956-08-06


In [6]:
df.head(1)

,数据层级,药品名称,靶点,作用机制,研发机构,疾病,全球最高研发阶段,全球最高研发阶段开始日期,全球研发状态,药品类别三,...,原研机构所在国家/地区,原研机构权益地区,中国最高研发阶段,美国最高研发阶段,中国最高研发阶段开始日期,美国最高研发阶段开始日期,highest_phase_standard,global_highest_phase_all_indication,global_highest_phase_all_indication_date,global_highest_phase_all_indication_standard
0,药品信息,杆菌肽,not available,多肽类抗生素,Columbia University(原研),感染;细菌感染;接触性皮炎(差异化疾病);葡萄球菌感染,批准上市,1948-07-29,Active,多肽,...,美国,全球;中国(内地);中国(港澳台);美国;欧洲;日本;其他,批准上市,批准上市,2008-09-22,1948-07-29,approved,批准上市,1948-07-29,approved


In [7]:
# Check current column names
print("Current columns:")
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")


Current columns:
0: 数据层级
1: 药品名称
2: 靶点
3: 作用机制
4: 研发机构
5: 疾病
6: 全球最高研发阶段
7: 全球最高研发阶段开始日期
8: 全球研发状态
9: 药品类别三
10: 药品类别二
11: 药品标签
12: 药理类型
13: 药品类别一
14: 药品异名
15: 载体类型
16: 疾病领域
17: 中国剂型
18: NPUID
19: 增补数据
20: 原研机构所在国家/地区
21: 原研机构权益地区
22: 中国最高研发阶段
23: 美国最高研发阶段
24: 中国最高研发阶段开始日期
25: 美国最高研发阶段开始日期
26: highest_phase_standard
27: global_highest_phase_all_indication
28: global_highest_phase_all_indication_date
29: global_highest_phase_all_indication_standard


In [8]:
# Rename columns to English snake_case
column_mapping = {
    '数据层级': 'data_level',
    '药品名称': 'drug_name',
    '靶点': 'targets',
    '作用机制': 'mechanism_of_action',
    '研发机构': 'research_institution',
    '疾病': 'indication',
    '全球最高研发阶段': 'global_highest_phase',
    '全球最高研发阶段开始日期': 'global_highest_phase_start_date',
    '全球研发状态': 'global_development_status',
    '全球研发状态开始日期': 'global_development_status_start_date',
    '药品类别三': 'drug_category_3',
    '药品类别二': 'drug_category_2',
    '药品类别一': 'drug_category_1',
    '药品异名': 'drug_synonyms',
    '药品标签': 'drug_tag',
    '药理类型': 'pharmacological_type',
    '疾病领域': 'disease_area',
    '载体类型': 'carrier_type',
    '原研机构所在国家/地区': 'originator_country',
    '原研机构权益地区': 'originator_rightsregion',
    '中国最高研发阶段': 'china_highest_phase',
    '美国最高研发阶段': 'usa_highest_phase',
    '中国最高研发阶段开始日期': 'china_highest_phase_start_date',
    '美国最高研发阶段开始日期': 'usa_highest_phase_start_date',
    'NPUID': 'npuid',
    '增补数据': 'additional_data',
    '中国剂型': 'formulation_china',
}

# Apply the renaming
df = df.rename(columns=column_mapping)

# Show the new column names
print("Updated columns:")
print(df.columns.tolist())


Updated columns:
['data_level', 'drug_name', 'targets', 'mechanism_of_action', 'research_institution', 'indication', 'global_highest_phase', 'global_highest_phase_start_date', 'global_development_status', 'drug_category_3', 'drug_category_2', 'drug_tag', 'pharmacological_type', 'drug_category_1', 'drug_synonyms', 'carrier_type', 'disease_area', 'formulation_china', 'npuid', 'additional_data', 'originator_country', 'originator_rightsregion', 'china_highest_phase', 'usa_highest_phase', 'china_highest_phase_start_date', 'usa_highest_phase_start_date', 'highest_phase_standard', 'global_highest_phase_all_indication', 'global_highest_phase_all_indication_date', 'global_highest_phase_all_indication_standard']


In [9]:
df = df.rename(columns = {
    'global_highest_phase': 'global_highest_phase_indication',
    'global_highest_phase_start_date': 'global_highest_phase_indication_start_date',
})

In [10]:
# Count research institutions per row
# '--' means not available (count as 0)
# Multiple institutions are separated by ';'

def count_institutions(value):
    if pd.isna(value) or value == '--':
        return 0
    # Split by ';' and count non-empty parts
    parts = [p.strip() for p in str(value).split(';') if p.strip() and p.strip() != '--']
    return len(parts)

df['research_institution_count'] = df['research_institution'].apply(count_institutions)

# Display the distribution of counts
print("Distribution of research institution counts:")
print(df['research_institution_count'].value_counts().sort_index())

print(f"\nTotal rows: {len(df)}")
print(f"Rows with '--' (0 institutions): {(df['research_institution_count'] == 0).sum()}")
print(f"Rows with 1 institution: {(df['research_institution_count'] == 1).sum()}")
print(f"Rows with multiple institutions (separated by ';'): {(df['research_institution_count'] > 1).sum()}")

Distribution of research institution counts:
research_institution_count
0      27
1    1252
2     519
3     172
4     146
5      15
6       6
Name: count, dtype: int64

Total rows: 2137
Rows with '--' (0 institutions): 27
Rows with 1 institution: 1252
Rows with multiple institutions (separated by ';'): 858


In [11]:
# Create is_cyclic_peptide column based on whether drug_tag contains '环肽'
df['is_cyclic_peptide'] = df['drug_tag'].str.contains('环肽', na=False)

# Verify the results
print(f"is_cyclic_peptide value counts:")
print(df['is_cyclic_peptide'].value_counts())

is_cyclic_peptide value counts:
is_cyclic_peptide
True    2137
Name: count, dtype: int64


In [12]:
df['is_rare_disease'] = df['disease_area'].str.contains('罕见疾病领域', na=False)

# Verify the results
print(f"is_rare_disease value counts:")
print(df['is_rare_disease'].value_counts())

is_rare_disease value counts:
is_rare_disease
False    1513
True      624
Name: count, dtype: int64


In [13]:
df['is_top20_mnc'] = df['research_institution'].str.lower().str.contains('top20', na=False)

In [14]:
# Extract Top20 MNC company names from research_institution
import re

def extract_top20_mnc(value):
    if pd.isna(value) or value == '--':
        return []
    
    top20_mncs = []
    # Split by ';' to get individual institutions
    institutions = [inst.strip() for inst in str(value).split(';')]
    
    for inst in institutions:
        if '(Top20 MNC)' in inst:
            # Extract company name - everything before (Top20 MNC)
            # Pattern: company_name(possibly with parenthetical like (AbbVie))(Top20 MNC)...
            match = re.match(r'^(.+?)\(Top20 MNC\)', inst)
            if match:
                company_name = match.group(1).strip()
                top20_mncs.append(company_name)
    
    return top20_mncs

# Apply to create new column
df['top20_mnc_list'] = df['research_institution'].apply(extract_top20_mnc)

# Verify with some examples
print("Examples of extracted Top20 MNC companies:")
# Get unique research_institution values where is_top20_mnc is True (avoid list column for drop_duplicates)
unique_institutions = df[df['is_top20_mnc'] == True]['research_institution'].drop_duplicates().head(20)
for inst in unique_institutions:
    mnc_list = extract_top20_mnc(inst)
    print(f"  {inst}")
    print(f"    -> {mnc_list}")
    print()

print(f"\nTotal rows with Top20 MNC: {df['top20_mnc_list'].apply(len).gt(0).sum()}")
print(f"Rows with multiple Top20 MNCs: {df['top20_mnc_list'].apply(len).gt(1).sum()}")


Examples of extracted Top20 MNC companies:
  Eli Lilly(Top20 MNC)(原研)
    -> ['Eli Lilly']

  Pfizer(Top20 MNC)(原研)
    -> ['Pfizer']

  Novartis(Top20 MNC)(原研)
    -> ['Novartis']

  Astellas Pharma(Top20 MNC);Daiichi Sankyo(原研)
    -> ['Astellas Pharma']

  Bayer(Top20 MNC);Hoechst(Sanofi)(Top20 MNC)(原研)(无权益)
    -> ['Bayer', 'Hoechst(Sanofi)']

  Novartis(Top20 MNC)(原研);Kacefa
    -> ['Novartis']

  Millennium Pharmaceuticals(Takeda Pharmaceuticals)(Top20 MNC)(原研);Schering-Plough(Merck & Co.)(Top20 MNC)
    -> ['Millennium Pharmaceuticals(Takeda Pharmaceuticals)', 'Schering-Plough(Merck & Co.)']

  Bayer(Top20 MNC)(原研)
    -> ['Bayer']

  Johnson & Johnson(Top20 MNC);Ferring Pharmaceuticals(原研)
    -> ['Johnson & Johnson']

  Merck & Co.(Top20 MNC)(原研)
    -> ['Merck & Co.']

  Merck KGaA(Top20 MNC)(原研);海森生物
    -> ['Merck KGaA']

  Sandoz;Astellas Pharma(Top20 MNC)(原研)(无权益)
    -> ['Astellas Pharma']

  Allergan(AbbVie)(Top20 MNC)(原研)
    -> ['Allergan(AbbVie)']

  Gilead Sciences(

In [15]:
# Extract originator companies (原研) from research_institution - returns a list

def extract_originators(value):
    if pd.isna(value) or value == '--':
        return []
    
    originators = []
    # Split by ';' to get individual institutions
    institutions = [inst.strip() for inst in str(value).split(';')]
    
    for inst in institutions:
        if '(原研)' in inst:
            # Extract company name - everything before (原研), but remove (Top20 MNC) if present
            # Pattern: company_name(possibly with parenthetical)(Top20 MNC)?(原研)
            match = re.match(r'^(.+?)\(原研\)', inst)
            if match:
                company_name = match.group(1).strip()
                # Remove (Top20 MNC) suffix if present
                company_name = re.sub(r'\(Top20 MNC\)$', '', company_name).strip()
                originators.append(company_name)
    
    return originators

# Apply to create new columns
df['originator_company_list'] = df['research_institution'].apply(extract_originators)
df['count_originator'] = df['originator_company_list'].apply(len)

# Verify with some examples
print("Examples of extracted originator companies:")
unique_institutions = df[df['originator_company_list'].apply(len) > 0]['research_institution'].drop_duplicates().head(20)
for inst in unique_institutions:
    originators = extract_originators(inst)
    print(f"  {inst}")
    print(f"    -> {originators}")
    print()

print(f"\nTotal rows with originator companies: {df['originator_company_list'].apply(len).gt(0).sum()}")
print(f"Rows with multiple originators: {df['originator_company_list'].apply(len).gt(1).sum()}")


Examples of extracted originator companies:
  Columbia University(原研)
    -> ['Columbia University']

  Travere Therapeutics(原研)
    -> ['Travere Therapeutics']

  Recordati Rare Diseases(原研);Tiziana Life Sciences;Rasna(无权益)
    -> ['Recordati Rare Diseases']

  Eli Lilly(Top20 MNC)(原研)
    -> ['Eli Lilly']

  Ferring Pharmaceuticals(原研);维健医药
    -> ['Ferring Pharmaceuticals']

  Par Pharmaceutical(Endo International)(原研)
    -> ['Par Pharmaceutical(Endo International)']

  Pfizer(Top20 MNC)(原研)
    -> ['Pfizer']

  Par Sterile Products(原研)
    -> ['Par Sterile Products']

  Sandoz(原研)
    -> ['Sandoz']

  Novartis(Top20 MNC)(原研)
    -> ['Novartis']

  Keenova Therapeutics(原研)
    -> ['Keenova Therapeutics']

  Astellas Pharma(Top20 MNC);Daiichi Sankyo(原研)
    -> ['Daiichi Sankyo']

  Bayer(Top20 MNC);Hoechst(Sanofi)(Top20 MNC)(原研)(无权益)
    -> ['Hoechst(Sanofi)']

  Novartis(Top20 MNC)(原研);Kacefa
    -> ['Novartis']

  Millennium Pharmaceuticals(Takeda Pharmaceuticals)(Top20 MNC)(原研);S

In [16]:
# Parse research_institution into a list by splitting on semicolon

def parse_institutions_list(value):
    if pd.isna(value) or value == '--':
        return []
    # Split by ';' and strip whitespace from each part
    parts = [p.strip() for p in str(value).split(';') if p.strip()]
    return parts

df['research_institution_list'] = df['research_institution'].apply(parse_institutions_list)

# Verify with some examples
print("Examples of parsed research institution lists:")
sample = df[df['research_institution_list'].apply(len) > 0][['research_institution', 'research_institution_list']].head(10)
for idx, row in sample.iterrows():
    print(f"  {row['research_institution']}")
    print(f"    -> {row['research_institution_list']}")
    print()


Examples of parsed research institution lists:
  Columbia University(原研)
    -> ['Columbia University(原研)']

  Columbia University(原研)
    -> ['Columbia University(原研)']

  Columbia University(原研)
    -> ['Columbia University(原研)']

  Columbia University(原研)
    -> ['Columbia University(原研)']

  Columbia University(原研)
    -> ['Columbia University(原研)']

  Travere Therapeutics(原研)
    -> ['Travere Therapeutics(原研)']

  Travere Therapeutics(原研)
    -> ['Travere Therapeutics(原研)']

  Travere Therapeutics(原研)
    -> ['Travere Therapeutics(原研)']

  Travere Therapeutics(原研)
    -> ['Travere Therapeutics(原研)']

  Recordati Rare Diseases(原研);Tiziana Life Sciences;Rasna(无权益)
    -> ['Recordati Rare Diseases(原研)', 'Tiziana Life Sciences', 'Rasna(无权益)']



In [17]:
# Create therapeutic_area_norm by removing '罕见疾病领域' from disease_area

def normalize_therapeutic_area(value):
    if pd.isna(value) or value == '--':
        return value
    # Remove '罕见疾病领域' and clean up
    normalized = str(value).replace('罕见疾病领域', '').strip()
    # Remove leading/trailing semicolons and clean up multiple semicolons
    normalized = ';'.join([p.strip() for p in normalized.split(';') if p.strip()])
    return normalized if normalized else None

df['therapeutic_area_norm'] = df['disease_area'].apply(normalize_therapeutic_area)

# Verify with examples where '罕见疾病领域' was present
print("Examples of therapeutic_area_norm (where 罕见疾病领域 was removed):")
has_rare = df[df['disease_area'].str.contains('罕见疾病领域', na=False)][['disease_area', 'therapeutic_area_norm']].drop_duplicates().head(15)
for idx, row in has_rare.iterrows():
    print(f"  {row['disease_area']}")
    print(f"    -> {row['therapeutic_area_norm']}")
    print()

print(f"\nRows containing '罕见疾病领域': {df['disease_area'].str.contains('罕见疾病领域', na=False).sum()}")


Examples of therapeutic_area_norm (where 罕见疾病领域 was removed):
  罕见疾病领域;肿瘤领域
    -> 肿瘤领域

  精神领域;其他领域;血液领域;罕见疾病领域;泌尿生殖领域;内分泌及代谢领域;感染领域;消化领域;心脑血管领域
    -> 精神领域;其他领域;血液领域;泌尿生殖领域;内分泌及代谢领域;感染领域;消化领域;心脑血管领域

  血液领域;罕见疾病领域
    -> 血液领域

  心脑血管领域;罕见疾病领域
    -> 心脑血管领域

  骨骼肌肉领域;免疫领域;皮肤领域;眼科领域;罕见疾病领域;泌尿生殖领域;血液领域;感染领域;消化领域;心脑血管领域
    -> 骨骼肌肉领域;免疫领域;皮肤领域;眼科领域;泌尿生殖领域;血液领域;感染领域;消化领域;心脑血管领域

  罕见疾病领域;免疫领域
    -> 免疫领域

  罕见疾病领域;心脑血管领域
    -> 心脑血管领域

  内分泌及代谢领域;罕见疾病领域;肿瘤领域;消化领域;其他领域;眼科领域;泌尿生殖领域;心脑血管领域;免疫领域;呼吸领域;血液领域
    -> 内分泌及代谢领域;肿瘤领域;消化领域;其他领域;眼科领域;泌尿生殖领域;心脑血管领域;免疫领域;呼吸领域;血液领域

  肿瘤领域;罕见疾病领域
    -> 肿瘤领域

  内分泌及代谢领域;罕见疾病领域
    -> 内分泌及代谢领域

  泌尿生殖领域;罕见疾病领域
    -> 泌尿生殖领域

  消化领域;罕见疾病领域
    -> 消化领域

  罕见疾病领域;肿瘤领域;呼吸领域
    -> 肿瘤领域;呼吸领域

  血液领域;肿瘤领域;罕见疾病领域
    -> 血液领域;肿瘤领域

  呼吸领域;罕见疾病领域
    -> 呼吸领域


Rows containing '罕见疾病领域': 624


In [18]:
# Rename indication to indication_original and create cleaned indication column

df = df.rename(columns={'indication': 'indication_original'})

def clean_indication(value):
    if pd.isna(value) or value == '--':
        return value
    # Remove (差异化疾病) and clean up
    cleaned = str(value).replace('(差异化疾病)', '').strip()
    return cleaned if cleaned else None

df['indication'] = df['indication_original'].apply(clean_indication)

# Verify with examples where (差异化疾病) was present
print("Examples of cleaned indication (where (差异化疾病) was removed):")
has_diff = df[df['indication_original'].str.contains('差异化疾病', na=False)][['indication_original', 'indication']].drop_duplicates().head(15)
for idx, row in has_diff.iterrows():
    print(f"  {row['indication_original']}")
    print(f"    -> {row['indication']}")
    print()

print(f"\nRows containing '(差异化疾病)': {df['indication_original'].str.contains('差异化疾病', na=False).sum()}")


Examples of cleaned indication (where (差异化疾病) was removed):


  感染;细菌感染;接触性皮炎(差异化疾病);葡萄球菌感染
    -> 感染;细菌感染;接触性皮炎;葡萄球菌感染

  接触性皮炎(差异化疾病)
    -> 接触性皮炎

  细菌性肺炎;菌血症(差异化疾病);细菌性心内膜炎(差异化疾病)
    -> 细菌性肺炎;菌血症;细菌性心内膜炎

  菌血症(差异化疾病)
    -> 菌血症

  细菌性心内膜炎(差异化疾病)
    -> 细菌性心内膜炎

  肺结核(差异化疾病)
    -> 肺结核

  肾盂肾炎(差异化疾病);A型血友病(差异化疾病);血管性血友病(差异化疾病);血小板减少症(差异化疾病);尿崩症;中枢性尿崩症;肾源性尿崩症(差异化疾病);低钠血症(差异化疾病);维生素D缺乏(差异化疾病);双相情感障碍(差异化疾病);遗尿症(差异化疾病);低血压(差异化疾病);动脉粥样硬化(差异化疾病);脑出血(差异化疾病);肝硬化(差异化疾病);膀胱过度活动症(差异化疾病);膀胱炎(差异化疾病);尿频(差异化疾病);子宫内膜异位(差异化疾病);前列腺疾病(差异化疾病);出血;外伤;月经过多(差异化疾病);钩端螺旋体病(差异化疾病);出血性卒中(差异化疾病);夜尿症(差异化疾病);术后出血;获得性血友病(差异化疾病);肾绞痛(差异化疾病)
    -> 肾盂肾炎;A型血友病;血管性血友病;血小板减少症;尿崩症;中枢性尿崩症;肾源性尿崩症;低钠血症;维生素D缺乏;双相情感障碍;遗尿症;低血压;动脉粥样硬化;脑出血;肝硬化;膀胱过度活动症;膀胱炎;尿频;子宫内膜异位;前列腺疾病;出血;外伤;月经过多;钩端螺旋体病;出血性卒中;夜尿症;术后出血;获得性血友病;肾绞痛

  肾盂肾炎(差异化疾病)
    -> 肾盂肾炎

  A型血友病(差异化疾病)
    -> A型血友病

  血管性血友病(差异化疾病)
    -> 血管性血友病

  血小板减少症(差异化疾病)
    -> 血小板减少症

  遗尿症(差异化疾病)
    -> 遗尿症

  肝硬化(差异化疾病)
    -> 肝硬化

  膀胱炎(差异化疾病)
    -> 膀胱炎

  尿频(差异化疾病)
    -> 尿频


Rows containing '(差异化疾病)': 585


In [19]:
# Parse therapeutic_area_norm into a list and add count

def parse_therapeutic_area_list(value):
    if pd.isna(value) or value == '--' or value == '':
        return []
    # Split by ';' and strip whitespace from each part
    parts = [p.strip() for p in str(value).split(';') if p.strip()]
    return parts

df['therapeutic_area_list'] = df['therapeutic_area_norm'].apply(parse_therapeutic_area_list)
df['count_therapeutic_area'] = df['therapeutic_area_list'].apply(len)

# Verify with examples
print("Examples of therapeutic_area_list:")
sample = df[df['count_therapeutic_area'] > 0][['therapeutic_area_norm', 'therapeutic_area_list', 'count_therapeutic_area']].drop_duplicates(subset=['therapeutic_area_norm']).head(15)
for idx, row in sample.iterrows():
    print(f"  {row['therapeutic_area_norm']}")
    print(f"    -> {row['therapeutic_area_list']} (count: {row['count_therapeutic_area']})")
    print()

print(f"\nDistribution of therapeutic area count:")
print(df['count_therapeutic_area'].value_counts().sort_index())


Examples of therapeutic_area_list:
  感染领域;皮肤领域
    -> ['感染领域', '皮肤领域'] (count: 2)

  感染领域
    -> ['感染领域'] (count: 1)

  皮肤领域
    -> ['皮肤领域'] (count: 1)

  泌尿生殖领域;精神领域
    -> ['泌尿生殖领域', '精神领域'] (count: 2)

  泌尿生殖领域
    -> ['泌尿生殖领域'] (count: 1)

  精神领域
    -> ['精神领域'] (count: 1)

  感染领域;呼吸领域;心脑血管领域
    -> ['感染领域', '呼吸领域', '心脑血管领域'] (count: 3)

  感染领域;呼吸领域
    -> ['感染领域', '呼吸领域'] (count: 2)

  感染领域;心脑血管领域
    -> ['感染领域', '心脑血管领域'] (count: 2)

  肿瘤领域
    -> ['肿瘤领域'] (count: 1)

  精神领域;其他领域;血液领域;泌尿生殖领域;内分泌及代谢领域;感染领域;消化领域;心脑血管领域
    -> ['精神领域', '其他领域', '血液领域', '泌尿生殖领域', '内分泌及代谢领域', '感染领域', '消化领域', '心脑血管领域'] (count: 8)

  感染领域;泌尿生殖领域
    -> ['感染领域', '泌尿生殖领域'] (count: 2)

  血液领域
    -> ['血液领域'] (count: 1)

  内分泌及代谢领域
    -> ['内分泌及代谢领域'] (count: 1)

  消化领域
    -> ['消化领域'] (count: 1)


Distribution of therapeutic area count:
count_therapeutic_area
0        8
1     1763
2      313
3       26
4       10
5        5
6        3
7        3
8        4
9        1
10       1
Name: count, dtype: int64


In [20]:
# Create therapeutic_area_norm (single value): prioritize '肿瘤领域', otherwise first value

def get_primary_therapeutic_area(area_list):
    if not area_list or len(area_list) == 0:
        return None
    # If contains '肿瘤领域', return that
    if '肿瘤领域' in area_list:
        return '肿瘤领域'
    # Otherwise return the first one
    return area_list[0]

df['therapeutic_area_norm'] = df['therapeutic_area_list'].apply(get_primary_therapeutic_area)

# Verify with examples where there are multiple therapeutic areas
print("Examples of therapeutic_area_norm (single value):")
multi_area = df[df['count_therapeutic_area'] > 1][['therapeutic_area_list', 'therapeutic_area_norm']].head(30)
seen = set()
count = 0
for idx, row in multi_area.iterrows():
    key = str(row['therapeutic_area_list'])
    if key not in seen and count < 15:
        seen.add(key)
        count += 1
        print(f"  {row['therapeutic_area_list']}")
        print(f"    -> {row['therapeutic_area_norm']}")
        print()

print(f"\nDistribution of therapeutic_area_norm:")
print(df['therapeutic_area_norm'].value_counts())

Examples of therapeutic_area_norm (single value):
  ['感染领域', '皮肤领域']
    -> 感染领域

  ['泌尿生殖领域', '精神领域']
    -> 泌尿生殖领域

  ['感染领域', '呼吸领域', '心脑血管领域']
    -> 感染领域

  ['感染领域', '呼吸领域']
    -> 感染领域

  ['感染领域', '心脑血管领域']
    -> 感染领域

  ['精神领域', '其他领域', '血液领域', '泌尿生殖领域', '内分泌及代谢领域', '感染领域', '消化领域', '心脑血管领域']
    -> 精神领域

  ['感染领域', '泌尿生殖领域']
    -> 感染领域

  ['感染领域', '消化领域']
    -> 感染领域

  ['其他领域', '内分泌及代谢领域']
    -> 其他领域

  ['骨骼肌肉领域', '免疫领域', '皮肤领域', '眼科领域', '泌尿生殖领域', '血液领域', '感染领域', '消化领域', '心脑血管领域']
    -> 骨骼肌肉领域

  ['骨骼肌肉领域', '免疫领域']
    -> 骨骼肌肉领域

  ['免疫领域', '皮肤领域']
    -> 免疫领域

  ['免疫领域', '泌尿生殖领域']
    -> 免疫领域

  ['眼科领域', '免疫领域']
    -> 眼科领域

  ['消化领域', '免疫领域']
    -> 消化领域


Distribution of therapeutic_area_norm:
therapeutic_area_norm
肿瘤领域        1089
心脑血管领域       135
感染领域         133
内分泌及代谢领域     111
诊断试剂领域       110
免疫领域          96
消化领域          81
泌尿生殖领域        66
其他领域          62
眼科领域          55
神经领域          51
呼吸领域          35
骨骼肌肉领域        28
精神领域          27
血液领域          23
麻醉镇痛领

In [21]:
# Create therapeutic_area_primary, secondary, tertiary from therapeutic_area_list

df['therapeutic_area_primary'] = df['therapeutic_area_list'].apply(lambda x: x[0] if len(x) > 0 else None)
df['therapeutic_area_secondary'] = df['therapeutic_area_list'].apply(lambda x: x[1] if len(x) > 1 else None)
df['therapeutic_area_tertiary'] = df['therapeutic_area_list'].apply(lambda x: x[2] if len(x) > 2 else None)

# Verify with examples
print("Examples of therapeutic_area primary/secondary/tertiary:")
sample = df[df['count_therapeutic_area'] >= 2][['therapeutic_area_list', 'therapeutic_area_primary', 'therapeutic_area_secondary', 'therapeutic_area_tertiary']].head(10)
for idx, row in sample.iterrows():
    print(f"  {row['therapeutic_area_list']}")
    print(f"    primary: {row['therapeutic_area_primary']}, secondary: {row['therapeutic_area_secondary']}, tertiary: {row['therapeutic_area_tertiary']}")
    print()

print(f"\nCounts:")
print(f"  Rows with primary: {df['therapeutic_area_primary'].notna().sum()}")
print(f"  Rows with secondary: {df['therapeutic_area_secondary'].notna().sum()}")
print(f"  Rows with tertiary: {df['therapeutic_area_tertiary'].notna().sum()}")


Examples of therapeutic_area primary/secondary/tertiary:
  ['感染领域', '皮肤领域']
    primary: 感染领域, secondary: 皮肤领域, tertiary: None

  ['泌尿生殖领域', '精神领域']
    primary: 泌尿生殖领域, secondary: 精神领域, tertiary: None

  ['感染领域', '呼吸领域', '心脑血管领域']
    primary: 感染领域, secondary: 呼吸领域, tertiary: 心脑血管领域

  ['感染领域', '呼吸领域']
    primary: 感染领域, secondary: 呼吸领域, tertiary: None

  ['感染领域', '心脑血管领域']
    primary: 感染领域, secondary: 心脑血管领域, tertiary: None

  ['精神领域', '其他领域', '血液领域', '泌尿生殖领域', '内分泌及代谢领域', '感染领域', '消化领域', '心脑血管领域']
    primary: 精神领域, secondary: 其他领域, tertiary: 血液领域

  ['感染领域', '泌尿生殖领域']
    primary: 感染领域, secondary: 泌尿生殖领域, tertiary: None

  ['感染领域', '消化领域']
    primary: 感染领域, secondary: 消化领域, tertiary: None

  ['感染领域', '消化领域']
    primary: 感染领域, secondary: 消化领域, tertiary: None

  ['其他领域', '内分泌及代谢领域']
    primary: 其他领域, secondary: 内分泌及代谢领域, tertiary: None


Counts:
  Rows with primary: 2129
  Rows with secondary: 366
  Rows with tertiary: 53


In [22]:
# Create sort value columns for phases

phase_sort_mapping = {
    'approved': 5,
    'pre-registration': 4,
    'phase 3': 3,
    'phase 2': 2,
    'phase 1': 1,
    'ind': 0,
    'pre-clinical': -1
}

df['global_highest_phase_all_indication_sort_val'] = df['global_highest_phase_all_indication_standard'].map(phase_sort_mapping)
df['highest_phase_standard_sort_val'] = df['highest_phase_standard'].map(phase_sort_mapping)

# Verify the mapping
print("global_highest_phase_all_indication_standard -> sort_val:")
print(df[['global_highest_phase_all_indication_standard', 'global_highest_phase_all_indication_sort_val']].drop_duplicates().sort_values('global_highest_phase_all_indication_sort_val'))

print("\nhighest_phase_standard -> sort_val:")
print(df[['highest_phase_standard', 'highest_phase_standard_sort_val']].drop_duplicates().sort_values('highest_phase_standard_sort_val'))


global_highest_phase_all_indication_standard -> sort_val:
     global_highest_phase_all_indication_standard  \
1409                                 pre-clinical   
1405                                          ind   
1200                                      phase 1   
921                                       phase 2   
736                                       phase 3   
713                              pre-registration   
0                                        approved   

      global_highest_phase_all_indication_sort_val  
1409                                            -1  
1405                                             0  
1200                                             1  
921                                              2  
736                                              3  
713                                              4  
0                                                5  

highest_phase_standard -> sort_val:
    highest_phase_standard  highest_phase_standard_sort_

In [23]:
# Map therapeutic area columns to English

therapeutic_area_en_mapping = {
    '感染领域': 'Infectious Diseases',
    '皮肤领域': 'Dermatology',
    '泌尿生殖领域': 'Genitourinary',
    '精神领域': 'Psychiatry',
    '肿瘤领域': 'Oncology',
    '血液领域': 'Hematology',
    '内分泌及代谢领域': 'Endocrinology & Metabolism',
    '消化领域': 'Digestive System',
    '其他领域': 'Others',
    '心脑血管领域': 'Cardiovascular & Cerebrovascular',
    '骨骼肌肉领域': 'Musculoskeletal',
    '免疫领域': 'Immunology',
    '眼科领域': 'Ophthalmology',
    '呼吸领域': 'Respiratory',
    '诊断试剂领域': 'Diagnostic Reagents',
    '麻醉镇痛领域': 'Anesthesia & Analgesia',
    '神经领域': 'Neurology'
}

df['therapeutic_area_norm_en'] = df['therapeutic_area_norm'].map(therapeutic_area_en_mapping)
df['therapeutic_area_primary_en'] = df['therapeutic_area_primary'].map(therapeutic_area_en_mapping)
df['therapeutic_area_secondary_en'] = df['therapeutic_area_secondary'].map(therapeutic_area_en_mapping)
df['therapeutic_area_tertiary_en'] = df['therapeutic_area_tertiary'].map(therapeutic_area_en_mapping)

# Verify the mapping
print("Therapeutic area English mapping:")
print(df[['therapeutic_area_norm', 'therapeutic_area_norm_en']].drop_duplicates().dropna())

# Check for any unmapped values
unmapped_norm = df[df['therapeutic_area_norm'].notna() & df['therapeutic_area_norm_en'].isna()]['therapeutic_area_norm'].unique()
unmapped_primary = df[df['therapeutic_area_primary'].notna() & df['therapeutic_area_primary_en'].isna()]['therapeutic_area_primary'].unique()
if len(unmapped_norm) > 0:
    print(f"\nUnmapped values in therapeutic_area_norm: {unmapped_norm}")
if len(unmapped_primary) > 0:
    print(f"\nUnmapped values in therapeutic_area_primary: {unmapped_primary}")


Therapeutic area English mapping:
    therapeutic_area_norm          therapeutic_area_norm_en
0                    感染领域               Infectious Diseases
3                    皮肤领域                       Dermatology
5                  泌尿生殖领域                     Genitourinary
8                    精神领域                        Psychiatry
13                   肿瘤领域                          Oncology
25                   血液领域                        Hematology
28               内分泌及代谢领域        Endocrinology & Metabolism
30                   消化领域                  Digestive System
33                   其他领域                            Others
39                 心脑血管领域  Cardiovascular & Cerebrovascular
60                 骨骼肌肉领域                   Musculoskeletal
63                   免疫领域                        Immunology
69                   眼科领域                     Ophthalmology
120                  呼吸领域                       Respiratory
165                诊断试剂领域               Diagnostic Reagents
285   

In [24]:
# Check distribution of originator counts (using count_originator from cell 13)

print("Distribution of originator count per row:")
print(df['count_originator'].value_counts().sort_index())

# Show examples with multiple originators if any
multiple_originators = df[df['count_originator'] > 1]['research_institution'].drop_duplicates()
if len(multiple_originators) > 0:
    print(f"\nExamples with multiple (原研) companies ({len(multiple_originators)} unique):")
    for inst in multiple_originators.head(10):
        print(f"  {inst}")
else:
    print("\nNo cases with multiple (原研) companies found.")


Distribution of originator count per row:
count_originator
0      27
1    2012
2      96
4       2
Name: count, dtype: int64

Examples with multiple (原研) companies (27 unique):
  Minapharm(原研);Rhein biotech(Top20 MNC)(原研)
  复星医药;Polyphor(Spexis)(原研);University of Zurich(原研)(无权益)
  Sanofi(Top20 MNC);Orano Med(原研)(无权益);RadioMedix(原研)(无权益)
  University of Queensland(原研);Alsonex;Promics(原研)
  Molecular Targeting Technologies(原研);University of Antwerp(原研)
  Novartis(Top20 MNC)(原研);PeptiDream(原研)
  Circle Pharma(原研);Vall d'Hebron Institute of Oncology(原研)
  PeptiDream(原研);RayzeBio(Bristol-Myers Squibb)(Top20 MNC)(原研)
  Nantes University Hospital(原研);Angers University(原研)
  PeptiDream(原研);Kawasaki Medical School(原研)


In [25]:
df_indication_level = df[df['data_level'] == '疾病信息'].copy()
df_indication_level = df_indication_level.drop(columns = ['data_level'])
df_indication_level = df_indication_level.rename(columns = {
    'highest_phase_standard': 'highest_phase_indication_standard',
})


In [26]:
# Check unique indications for translation planning

unique_indications = df_indication_level['indication'].dropna().unique()
print(f"Number of unique indications: {len(unique_indications)}")
print(f"\nSample indications:")
for ind in unique_indications[:30]:
    print(f"  {ind}")


Number of unique indications: 560

Sample indications:
  细菌感染
  感染
  接触性皮炎
  葡萄球菌感染
  催产/引产
  产后子宫出血
  孤独症谱系障碍
  细菌性肺炎
  菌血症
  细菌性心内膜炎
  肾母细胞瘤
  横纹肌肉瘤
  睾丸癌
  尤文氏肉瘤
  妊娠滋养细胞肿瘤
  肺结核
  肾盂肾炎
  A型血友病
  血管性血友病
  血小板减少症
  中枢性尿崩症
  遗尿症
  肝硬化
  膀胱炎
  尿频
  出血
  夜尿症
  外伤
  低钠血症
  肾绞痛


In [28]:
# Translate indications from Chinese to English
# First install: pip install deep-translator

from deep_translator import GoogleTranslator
from tqdm import tqdm

# Create a translation cache to avoid re-translating duplicates
unique_indications = df_indication_level['indication'].dropna().unique()
translation_cache = {}

print(f"Translating {len(unique_indications)} unique indications...")

# Translate each unique indication
for ind in tqdm(unique_indications):
    if ind and ind not in translation_cache:
        try:
            translated = GoogleTranslator(source='zh-CN', target='en').translate(ind)
            translation_cache[ind] = translated
        except Exception as e:
            print(f"Error translating '{ind}': {e}")
            translation_cache[ind] = ind  # Keep original if translation fails

# Apply translation to create new column
df_indication_level['indication_en'] = df_indication_level['indication'].map(translation_cache)

print(f"\nTranslation complete!")
print(f"Sample translations:")
sample = df_indication_level[['indication', 'indication_en']].drop_duplicates().head(20)
for idx, row in sample.iterrows():
    print(f"  {row['indication']} -> {row['indication_en']}")


Translating 560 unique indications...


100%|██████████| 560/560 [03:30<00:00,  2.66it/s]


Translation complete!
Sample translations:
  细菌感染 -> bacterial infection
  感染 -> Infect
  接触性皮炎 -> contact dermatitis
  葡萄球菌感染 -> Staph infection
  催产/引产 -> Oxygen/induced labor
  产后子宫出血 -> Postpartum uterine bleeding
  孤独症谱系障碍 -> autism spectrum disorder
  细菌性肺炎 -> bacterial pneumonia
  菌血症 -> bacteremia
  细菌性心内膜炎 -> bacterial endocarditis
  肾母细胞瘤 -> nephroblastoma
  横纹肌肉瘤 -> rhabdomyosarcoma
  睾丸癌 -> testicular cancer
  尤文氏肉瘤 -> Ewing's sarcoma
  妊娠滋养细胞肿瘤 -> gestational trophoblastic tumor
  肺结核 -> tuberculosis
  肾盂肾炎 -> Pyelonephritis
  A型血友病 -> Hemophilia A
  血管性血友病 -> von Willebrand disease
  血小板减少症 -> Thrombocytopenia


In [29]:
df_indication_level.shape

(1630, 51)

## Get English information

In [30]:
df_en = pd.read_excel("../data/pharmcube_cyclic_peptide_20251225_en.xlsx", sheet_name=0)

In [31]:
df_en.columns


Index(['Data Level', 'Drug Name', 'Target', 'Action', 'Research Institute',
       'Disease', 'Global Highest Phase', 'Start Date of Global Highest Phase',
       'Global Status', 'Drug Class III', 'Drug Class II', 'Drug Tag', 'MoA',
       'Drug Name Synonym', 'Carrier Type', 'Disease Area', 'NPUID',
       'China Dosage Form', 'Originator Location', 'Territory of Originator',
       'CN Highest Phase', 'US Highest Phase',
       'Start Date of CN Highest Phase', 'Start Date of US Highest Phase'],
      dtype='object')

In [32]:
# Get Drug Name and NPUID from df_en and deduplicate

df_drug_npuid = df_en[['Drug Name', 'NPUID', 'Drug Name Synonym']].drop_duplicates()

print(f"Original rows in df_en: {len(df_en)}")
print(f"Deduplicated Drug Name + NPUID pairs: {len(df_drug_npuid)}")
df_drug_npuid.head(10)

Original rows in df_en: 2137
Deduplicated Drug Name + NPUID pairs: 507


,Drug Name,NPUID,Drug Name Synonym
0,bacitracin,DR004636,杆菌肽;bacitracin
5,oxytocin,DR032561,缩宫素;oxytocin;催产素;Syntocinon;TTA-121;OT;Pitocin
9,polymyxin B,DR049372,多粘菌素B;polymyxin B
13,dactinomycin,DR001704,放线菌素D;dactinomycin;Cosmegen;actinomycin D;Act D
19,capreomycin,DR023297,卷曲霉素;capreomycin;卷须霉素;结核霉素;Capastat
21,bacitracin+polymyxin B,DR066181,杆菌肽+多粘菌素B;bacitracin+polymyxin B
23,desmopressin,DR032515,去氨加压素;desmopressin;Ddavp;Concentraid;desmopres...
53,colistin,DR049240,粘菌素;colistin;抗敌素;COL
55,vasopressin,DR001309,血管加压素;vasopressin;arginine vasopressin;精氨酸加压素;...
58,viomycin,DR051819,紫霉素;viomycin


In [37]:
df_drug_npuid_renamed = df_drug_npuid.rename(columns={
    'Drug Name': 'drug_name_en', 
    'NPUID': 'npuid',
    'Drug Name Synonym': 'drug_synonyms_en'
})

In [38]:
df_drug_npuid_renamed.head()

,drug_name_en,npuid,drug_synonyms_en
0,bacitracin,DR004636,杆菌肽;bacitracin
5,oxytocin,DR032561,缩宫素;oxytocin;催产素;Syntocinon;TTA-121;OT;Pitocin
9,polymyxin B,DR049372,多粘菌素B;polymyxin B
13,dactinomycin,DR001704,放线菌素D;dactinomycin;Cosmegen;actinomycin D;Act D
19,capreomycin,DR023297,卷曲霉素;capreomycin;卷须霉素;结核霉素;Capastat


In [39]:
df_indication_level_en = df_indication_level.merge(df_drug_npuid_renamed, on='npuid', how='left')

In [40]:
df_indication_level_en.head()

,drug_name,targets,mechanism_of_action,research_institution,indication_original,global_highest_phase_indication,global_highest_phase_indication_start_date,global_development_status,drug_category_3,drug_category_2,...,therapeutic_area_tertiary,global_highest_phase_all_indication_sort_val,highest_phase_standard_sort_val,therapeutic_area_norm_en,therapeutic_area_primary_en,therapeutic_area_secondary_en,therapeutic_area_tertiary_en,indication_en,drug_name_en,drug_synonyms_en
0,杆菌肽,not available,多肽类抗生素,Columbia University(原研),细菌感染,批准上市,NaN,Active,多肽,化药,...,None,5,5,Infectious Diseases,Infectious Diseases,NaN,NaN,bacterial infection,bacitracin,杆菌肽;bacitracin
1,杆菌肽,not available,多肽类抗生素,Columbia University(原研),感染,III期临床,2009-10-28,Inactive,多肽,化药,...,None,5,3,Infectious Diseases,Infectious Diseases,NaN,NaN,Infect,bacitracin,杆菌肽;bacitracin
2,杆菌肽,not available,多肽类抗生素,Columbia University(原研),接触性皮炎(差异化疾病),II期临床,2005-08-22,Inactive,多肽,化药,...,None,5,2,Dermatology,Dermatology,NaN,NaN,contact dermatitis,bacitracin,杆菌肽;bacitracin
3,杆菌肽,not available,多肽类抗生素,Columbia University(原研),葡萄球菌感染,I期临床,2009-10-19,Inactive,多肽,化药,...,None,5,1,Infectious Diseases,Infectious Diseases,NaN,NaN,Staph infection,bacitracin,杆菌肽;bacitracin
4,缩宫素,oxytocin,合成的oxytocin,Travere Therapeutics(原研),催产/引产,批准上市,NaN,Active,多肽,化药,...,None,5,5,Genitourinary,Genitourinary,NaN,NaN,Oxygen/induced labor,oxytocin,缩宫素;oxytocin;催产素;Syntocinon;TTA-121;OT;Pitocin


In [41]:
df_indication_level_en.columns

Index(['drug_name', 'targets', 'mechanism_of_action', 'research_institution',
       'indication_original', 'global_highest_phase_indication',
       'global_highest_phase_indication_start_date',
       'global_development_status', 'drug_category_3', 'drug_category_2',
       'drug_tag', 'pharmacological_type', 'drug_category_1', 'drug_synonyms',
       'carrier_type', 'disease_area', 'formulation_china', 'npuid',
       'additional_data', 'originator_country', 'originator_rightsregion',
       'china_highest_phase', 'usa_highest_phase',
       'china_highest_phase_start_date', 'usa_highest_phase_start_date',
       'highest_phase_indication_standard',
       'global_highest_phase_all_indication',
       'global_highest_phase_all_indication_date',
       'global_highest_phase_all_indication_standard',
       'research_institution_count', 'is_cyclic_peptide', 'is_rare_disease',
       'is_top20_mnc', 'top20_mnc_list', 'originator_company_list',
       'count_originator', 'research_insti

In [42]:
cols_to_keep_drug_level = [
    'drug_name', 
    'drug_name_en',
    'drug_synonyms',
    'indication',
    'indication_en', 
    'targets', 
    'mechanism_of_action', 
    'research_institution',
    'drug_category_1', 
    'drug_category_2',
    'drug_category_3', 
    'drug_tag', 
    'pharmacological_type', 
    'indication_original', 
    'global_highest_phase_indication',
    'global_highest_phase_indication_start_date',
    'global_development_status', 
    'disease_area', 
    'npuid',
    'originator_country', 
    'highest_phase_indication_standard',
    'global_highest_phase_all_indication',
    'global_highest_phase_all_indication_date',
    'global_highest_phase_all_indication_standard',
    'research_institution_count', 
    'is_cyclic_peptide', 
    'is_rare_disease',
    'is_top20_mnc', 
    'top20_mnc_list', 
    'originator_company_list',
    'count_originator', 
    'research_institution_list',
    'therapeutic_area_norm', 
    'therapeutic_area_list',
    'count_therapeutic_area', 
    'therapeutic_area_primary',
    'therapeutic_area_secondary', 
    'therapeutic_area_tertiary',
    'global_highest_phase_all_indication_sort_val',
    'highest_phase_standard_sort_val', 
    'therapeutic_area_norm_en',
    'therapeutic_area_primary_en', 
    'therapeutic_area_secondary_en',
    'therapeutic_area_tertiary_en', 
    
]

# Create the new dataframe with only the selected columns
# Assuming your original dataframe is named 'df'
df_indication_level_en_clean = df_indication_level_en[cols_to_keep_drug_level].copy()

In [60]:
df_indication_level_en_clean_active = df_indication_level_en_clean[df_indication_level_en_clean['global_development_status'] == 'Active']

In [50]:
df_indication_level_en_clean_active.shape

(1180, 44)

In [53]:
# Expand dataframe: one row per company (indication-drug-company mapping)

# Explode the research_institution_list to create one row per company
df_indication_drug_company = df_indication_level_en_clean_active.explode('research_institution_list')

# Rename the exploded column to indicate it's a single company
df_indication_drug_company = df_indication_drug_company.rename(columns={'research_institution_list': 'company'})

print(f"Original df_indication_level rows: {len(df_indication_level)}")
print(f"Expanded df_indication_drug_company rows: {len(df_indication_drug_company)}")
print(f"Expansion factor: {len(df_indication_drug_company) / len(df_indication_level):.2f}x")

# Show sample
print(f"\nSample of expanded dataframe:")
df_indication_drug_company[['drug_name', 'indication', 'company']].head(15)


Original df_indication_level rows: 1630
Expanded df_indication_drug_company rows: 1892
Expansion factor: 1.16x

Sample of expanded dataframe:


,drug_name,indication,company
0,杆菌肽,细菌感染,Columbia University(原研)
4,缩宫素,催产/引产,Travere Therapeutics(原研)
5,缩宫素,产后子宫出血,Travere Therapeutics(原研)
6,缩宫素,孤独症谱系障碍,Travere Therapeutics(原研)
7,多粘菌素B,细菌性肺炎,NaN
8,多粘菌素B,菌血症,NaN
9,多粘菌素B,细菌性心内膜炎,NaN
10,放线菌素D,肾母细胞瘤,Recordati Rare Diseases(原研)
10,放线菌素D,肾母细胞瘤,Tiziana Life Sciences
10,放线菌素D,肾母细胞瘤,Rasna(无权益)


In [54]:
# Add is_originator_indication_drug and is_top20mnc_indication_drug columns

df_indication_drug_company['is_originator_indication_drug'] = df_indication_drug_company['company'].str.contains('(原研)', na=False, regex=False)
df_indication_drug_company['is_top20mnc_indication_drug'] = df_indication_drug_company['company'].str.contains('(Top20 MNC)', na=False, regex=False)

print("Value counts for is_originator_indication_drug:")
print(df_indication_drug_company['is_originator_indication_drug'].value_counts())

print("\nValue counts for is_top20mnc_indication_drug:")
print(df_indication_drug_company['is_top20mnc_indication_drug'].value_counts())

# Show sample
print("\nSample rows:")
df_indication_drug_company[['company', 'is_originator_indication_drug', 'is_top20mnc_indication_drug']].drop_duplicates().head(15)


Value counts for is_originator_indication_drug:
is_originator_indication_drug
True     1224
False     668
Name: count, dtype: int64

Value counts for is_top20mnc_indication_drug:
is_top20mnc_indication_drug
False    1560
True      332
Name: count, dtype: int64

Sample rows:


,company,is_originator_indication_drug,is_top20mnc_indication_drug
0,Columbia University(原研),True,False
4,Travere Therapeutics(原研),True,False
7,NaN,False,False
10,Recordati Rare Diseases(原研),True,False
10,Tiziana Life Sciences,False,False
10,Rasna(无权益),False,False
15,Eli Lilly(Top20 MNC)(原研),True,True
17,Ferring Pharmaceuticals(原研),True,False
17,维健医药,False,False
46,Par Pharmaceutical(Endo International)(原研),True,False


In [65]:
# Summarize df_indication_level_en_clean_active to drug level: one row per drug
# - Aggregate indication, therapeutic_area, and highest_phase into a JSON structure
# - Keep only the highest phase (max highest_phase_standard_sort_val) for other columns

import json

def aggregate_indications_to_json(group):
    """Aggregate indications into a JSON list with indication, therapeutic_area, and phase"""
    indications_list = []
    for _, row in group.iterrows():
        indication_obj = {
            'indication': row['indication'] if pd.notna(row['indication']) else None,
            'indication_en': row['indication_en'] if pd.notna(row['indication_en']) else None,
            'therapeutic_area': row['therapeutic_area_norm'] if pd.notna(row['therapeutic_area_norm']) else None,
            'therapeutic_area_en': row['therapeutic_area_norm_en'] if pd.notna(row['therapeutic_area_norm_en']) else None,
            'highest_phase': row['highest_phase_indication_standard'] if pd.notna(row['highest_phase_indication_standard']) else None,
            'highest_phase_sort_val': int(row['highest_phase_standard_sort_val']) if pd.notna(row['highest_phase_standard_sort_val']) else None
        }
        indications_list.append(indication_obj)
    return json.dumps(indications_list, ensure_ascii=False)

# Get the row with highest phase for each drug (for all other columns)
idx_max_phase = df_indication_level_en_clean_active.groupby('drug_name_en')['highest_phase_standard_sort_val'].idxmax()
df_drug_level_active = df_indication_level_en_clean_active.loc[idx_max_phase].copy()

# Aggregate indications into JSON for each drug
indications_json_agg = df_indication_level_en_clean_active.groupby('drug_name_en').apply(aggregate_indications_to_json).reset_index()
indications_json_agg.columns = ['drug_name_en', 'indications_json']

# Merge the aggregated indications JSON back to df_drug_level_active
df_drug_level_active = df_drug_level_active.drop(columns=['indication', 'indication_en', 'indication_original', 'therapeutic_area_norm', 'therapeutic_area_norm_en'], errors='ignore')
df_drug_level_active = df_drug_level_active.merge(indications_json_agg, on='drug_name_en', how='left')

print(f"Original rows (active indications): {len(df_indication_level_en_clean_active)}")
print(f"Summarized drug-level rows: {len(df_drug_level_active)}")
print(f"\nSample of combined indications (JSON structure):")
sample = df_drug_level_active[['drug_name_en', 'indications_json', 'highest_phase_indication_standard', 'highest_phase_standard_sort_val']].head(5)
for idx, row in sample.iterrows():
    print(f"\n  Drug: {row['drug_name_en']}")
    print(f"  Highest phase (drug level): {row['highest_phase_indication_standard']} (sort val: {row['highest_phase_standard_sort_val']})")
    # Pretty print the JSON
    indications = json.loads(row['indications_json'])
    print(f"  Indications ({len(indications)} total):")
    for ind in indications[:3]:  # Show first 3
        print(f"    - {ind['indication_en']} | {ind['therapeutic_area_en']} | {ind['highest_phase']}")

Original rows (active indications): 1180
Summarized drug-level rows: 504

Sample of combined indications (JSON structure):

  Drug: 111In-DPI-4452
  Highest phase (drug level): pre-clinical (sort val: -1)
  Indications (2 total):
    - cancer | Oncology | pre-clinical
    - SPECT imaging | Diagnostic Reagents | pre-clinical

  Drug: 111In-DPI-4501
  Highest phase (drug level): pre-clinical (sort val: -1)
  Indications (3 total):
    - renal cell carcinoma | Oncology | pre-clinical
    - SPECT imaging | Diagnostic Reagents | pre-clinical
    - colorectal cancer | Oncology | pre-clinical

  Drug: 111In-FAP-2286
  Highest phase (drug level): pre-clinical (sort val: -1)
  Indications (2 total):
    - solid tumors | Oncology | pre-clinical
    - SPECT imaging | Diagnostic Reagents | pre-clinical

  Drug: 13N-oxytocin
  Highest phase (drug level): phase 1 (sort val: 1)
  Indications (1 total):
    - PET imaging | Diagnostic Reagents | phase 1

  Drug: 161Tb-DOTA-LM3
  Highest phase (drug lev

C:\Users\Felix Fu\AppData\Local\Temp\ipykernel_3132\1412510252.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  indications_json_agg = df_indication_level_en_clean_active.groupby('drug_name_en').apply(aggregate_indications_to_json).reset_index()


In [66]:
# Check the resulting drug-level dataframe
print("Columns in df_drug_level_active:")
for col in df_drug_level_active.columns:
    print(f"  - {col}")

print(f"\nShape: {df_drug_level_active.shape}")
df_drug_level_active.head()

Columns in df_drug_level_active:
  - drug_name
  - drug_name_en
  - drug_synonyms
  - targets
  - mechanism_of_action
  - research_institution
  - drug_category_1
  - drug_category_2
  - drug_category_3
  - drug_tag
  - pharmacological_type
  - global_highest_phase_indication
  - global_highest_phase_indication_start_date
  - global_development_status
  - disease_area
  - npuid
  - originator_country
  - highest_phase_indication_standard
  - global_highest_phase_all_indication
  - global_highest_phase_all_indication_date
  - global_highest_phase_all_indication_standard
  - research_institution_count
  - is_cyclic_peptide
  - is_rare_disease
  - is_top20_mnc
  - top20_mnc_list
  - originator_company_list
  - count_originator
  - research_institution_list
  - therapeutic_area_list
  - count_therapeutic_area
  - therapeutic_area_primary
  - therapeutic_area_secondary
  - therapeutic_area_tertiary
  - global_highest_phase_all_indication_sort_val
  - highest_phase_standard_sort_val
  - ther

,drug_name,drug_name_en,drug_synonyms,targets,mechanism_of_action,research_institution,drug_category_1,drug_category_2,drug_category_3,drug_tag,...,count_therapeutic_area,therapeutic_area_primary,therapeutic_area_secondary,therapeutic_area_tertiary,global_highest_phase_all_indication_sort_val,highest_phase_standard_sort_val,therapeutic_area_primary_en,therapeutic_area_secondary_en,therapeutic_area_tertiary_en,indications_json
0,111In-DPI-4452,111In-DPI-4452,111In-DPI-4452,CAIX,含铟放射性示踪剂;anti-CAIX多肽偶联核素,Debiopharm;3B Pharmaceuticals(原研),创新药,化药,多肽;放射性药物;偶联药物,环肽;诊断用放射性药物;Potential First-in-Class,...,1,肿瘤领域,None,None,-1,-1,Oncology,NaN,NaN,"[{""indication"": ""癌症"", ""indication_en"": ""cancer..."
1,111In-DPI-4501,111In-DPI-4501,111In-DPI-4501,CAIX,含铟放射性示踪剂;anti-CAIX多肽偶联核素,Debiopharm;3B Pharmaceuticals(原研),创新药,化药,多肽;放射性药物;偶联药物,环肽;差异化疾病;诊断用放射性药物;Potential First-in-Class,...,1,肿瘤领域,None,None,-1,-1,Oncology,NaN,NaN,"[{""indication"": ""肾细胞癌"", ""indication_en"": ""rena..."
2,111In-FAP-2286,111In-FAP-2286,111In-FAP-2286,FAP,含铟放射性示踪剂;anti-FAP多肽偶联核素,Novartis(Top20 MNC);3B Pharmaceuticals(原研);Clo...,创新药,化药,多肽;放射性药物;偶联药物,环肽;诊断用放射性药物;中国无申报;Potential First-in-Class,...,1,肿瘤领域,None,None,-1,-1,Oncology,NaN,NaN,"[{""indication"": ""实体瘤"", ""indication_en"": ""solid..."
3,13N-oxytocin,13N-oxytocin,13N-oxytocin,OXTR,13N标记的PET药物;anti-OXTR多肽偶联核素,Tonix Pharmaceuticals(原研),创新药,化药,多肽;放射性药物;偶联药物,环肽;诊断用放射性药物;G蛋白偶联受体;Potential First-in-Class,...,1,诊断试剂领域,None,None,1,1,Diagnostic Reagents,NaN,NaN,"[{""indication"": ""PET显像"", ""indication_en"": ""PET..."
4,161Tb-DOTA-LM3,161Tb-DOTA-LM3,161Tb-DOTA-LM3,SSTR2,161Tb标记的放射性药物;anti-SSTR2多肽偶联核素,Paul Scherrer Institute(原研),创新药,化药,多肽;放射性药物;偶联药物,环肽;Potential First-in-Class,...,1,肿瘤领域,None,None,1,1,Oncology,NaN,NaN,"[{""indication"": ""胃肠胰神经内分泌肿瘤"", ""indication_en"":..."


In [68]:
# Save the drug-level summary
df_drug_level_active.to_csv('../data/cyclic_peptide_drug_level_active.csv', index=False, encoding='utf-8-sig')
print("Saved df_drug_level_active to ../data/cyclic_peptide_drug_level_active.csv")

Saved df_drug_level_active to ../data/cyclic_peptide_drug_level_active.csv


In [55]:
# Create clean company name by removing tags

def clean_company_name(value):
    if pd.isna(value) or value == '':
        return value
    cleaned = str(value)
    # Remove tags
    cleaned = cleaned.replace('(Top20 MNC)', '')
    cleaned = cleaned.replace('(原研)', '')
    cleaned = cleaned.replace('(无权益)', '')
    return cleaned.strip()

df_indication_drug_company['company_clean'] = df_indication_drug_company['company'].apply(clean_company_name)

# Show sample
print("Examples of cleaned company names:")
sample = df_indication_drug_company[['company', 'company_clean']].drop_duplicates().head(20)
for idx, row in sample.iterrows():
    if row['company'] != row['company_clean']:
        print(f"  {row['company']}")
        print(f"    -> {row['company_clean']}")
        print()

print(f"\nUnique companies before cleaning: {df_indication_drug_company['company'].nunique()}")
print(f"Unique companies after cleaning: {df_indication_drug_company['company_clean'].nunique()}")


Examples of cleaned company names:
  Columbia University(原研)
    -> Columbia University

  Travere Therapeutics(原研)
    -> Travere Therapeutics

  nan
    -> nan

  Recordati Rare Diseases(原研)
    -> Recordati Rare Diseases

  Rasna(无权益)
    -> Rasna

  Eli Lilly(Top20 MNC)(原研)
    -> Eli Lilly

  Ferring Pharmaceuticals(原研)
    -> Ferring Pharmaceuticals

  Par Pharmaceutical(Endo International)(原研)
    -> Par Pharmaceutical(Endo International)

  Pfizer(Top20 MNC)(原研)
    -> Pfizer

  Par Sterile Products(原研)
    -> Par Sterile Products

  Sandoz(原研)
    -> Sandoz

  Novartis(Top20 MNC)(原研)
    -> Novartis

  Keenova Therapeutics(原研)
    -> Keenova Therapeutics

  Astellas Pharma(Top20 MNC)
    -> Astellas Pharma

  Daiichi Sankyo(原研)
    -> Daiichi Sankyo

  Bayer(Top20 MNC)
    -> Bayer

  Hoechst(Sanofi)(Top20 MNC)(原研)(无权益)
    -> Hoechst(Sanofi)


Unique companies before cleaning: 439
Unique companies after cleaning: 403


In [56]:
df_indication_drug_company.columns

Index(['drug_name', 'drug_name_en', 'drug_synonyms', 'indication',
       'indication_en', 'targets', 'mechanism_of_action',
       'research_institution', 'drug_category_1', 'drug_category_2',
       'drug_category_3', 'drug_tag', 'pharmacological_type',
       'indication_original', 'global_highest_phase_indication',
       'global_highest_phase_indication_start_date',
       'global_development_status', 'disease_area', 'npuid',
       'originator_country', 'highest_phase_indication_standard',
       'global_highest_phase_all_indication',
       'global_highest_phase_all_indication_date',
       'global_highest_phase_all_indication_standard',
       'research_institution_count', 'is_cyclic_peptide', 'is_rare_disease',
       'is_top20_mnc', 'top20_mnc_list', 'originator_company_list',
       'count_originator', 'company', 'therapeutic_area_norm',
       'therapeutic_area_list', 'count_therapeutic_area',
       'therapeutic_area_primary', 'therapeutic_area_secondary',
       'therapeu

In [57]:
cols_to_keep = [
    'drug_name', 
    'drug_name_en', 
    'indication',
    'indication_en', 
    'company_clean',
    
    'therapeutic_area_norm',
    'therapeutic_area_norm_en',
    'highest_phase_indication_standard',
    'highest_phase_standard_sort_val', 
    
    'drug_synonyms', 
    'targets', 
    'mechanism_of_action', 
    'global_highest_phase_all_indication_standard',
    'global_highest_phase_all_indication_sort_val',
    'global_highest_phase_all_indication_date',
    'global_highest_phase_indication',
    'global_highest_phase_indication_start_date',
    'global_development_status',
    'drug_category_1', 
    'drug_category_2', 
    'drug_category_3',
    'drug_tag',
    'pharmacological_type',
    'is_top20_mnc', 
    'is_cyclic_peptide', 
    'is_rare_disease',
    'is_originator_indication_drug',
    'is_top20mnc_indication_drug', 
    'research_institution',
    'research_institution_count', 

    # 'originator_company_list',
    # 'count_originator',
    # 'top20_mnc_list', 
    # 'company', 
    # 'indication_original',      
    # 'disease_area', 
    # 'therapeutic_area_list', 'count_therapeutic_area',
    # 'therapeutic_area_primary', 'therapeutic_area_secondary',
    # 'therapeutic_area_tertiary',
    # 'therapeutic_area_primary_en', 'therapeutic_area_secondary_en',
    # 'therapeutic_area_tertiary_en', 
    # 'formulation_china', 
    # 'npuid',
    # 'additional_data', 
    # 'originator_country', 
    # 'originator_rightsregion',
    # 'china_highest_phase', 
    # 'china_highest_phase_start_date', 
    # 'usa_highest_phase',
    # 'usa_highest_phase_start_date',
    # 'global_highest_phase_all_indication',
]

# Create the new dataframe with only the selected columns
# Assuming your original dataframe is named 'df'
df_indication_drug_company_clean = df_indication_drug_company[cols_to_keep].copy()

In [59]:
df_indication_level_en.to_csv('../data/cyclic_peptide_indication_level.csv', index=False, encoding='utf-8-sig')
df_indication_level_en_clean.to_csv('../data/cyclic_peptide_indication_level_clean.csv', index=False, encoding='utf-8-sig')
df_indication_level_en_clean_active.to_csv('../data/cyclic_peptide_indication_level_clean_active.csv', index=False, encoding='utf-8-sig')
df_indication_drug_company.to_csv('../data/cyclic_peptide_drug_indication_company.csv', index=False, encoding='utf-8-sig')
df_indication_drug_company_clean.to_csv('../data/cyclic_peptide_drug_indication_company_clean.csv', index=False, encoding='utf-8-sig')